4. VERİ ÖNİŞLEME

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import sklearn as sk

In [2]:
data_orj=pd.read_csv('insurance.csv')
df= data_orj.copy()

In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()

df['smoker'] = label_encoder.fit_transform(df['smoker']) 
df['region']=label_encoder.fit_transform(df['region'])
df['sex']=label_encoder.fit_transform(df['sex'])

In [4]:
df.corr()['charges'].sort_values(ascending=False)

charges     1.000000
smoker      0.787251
age         0.299008
bmi         0.198341
children    0.067998
sex         0.057292
region     -0.006208
Name: charges, dtype: float64

Charges ile korelasyonu 0.2 den düşük olan değişkenleri almadan regresyon modellerini oluşturabiliriz  

In [5]:
df_new=df[['age','smoker','bmi','charges']]
df_new

,age,smoker,bmi,charges
0,19,1,27.900,16884.92400
1,18,0,33.770,1725.55230
2,28,0,33.000,4449.46200
3,33,0,22.705,21984.47061
4,32,0,28.880,3866.85520
...,...,...,...,...
1333,50,0,30.970,10600.54830
1334,18,0,31.920,2205.98080
1335,18,0,36.850,1629.83350
1336,21,0,25.800,2007.94500


Veriyi normalize ediyoruz

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df_new)

normalized_df_new = pd.DataFrame(normalized_data, columns=df_new.columns)

print(normalized_df_new)

           age  smoker       bmi   charges
0     0.021739     1.0  0.321227  0.251611
1     0.000000     0.0  0.479150  0.009636
2     0.217391     0.0  0.458434  0.053115
3     0.326087     0.0  0.181464  0.333010
4     0.304348     0.0  0.347592  0.043816
...        ...     ...       ...       ...
1333  0.695652     0.0  0.403820  0.151299
1334  0.000000     0.0  0.429379  0.017305
1335  0.000000     0.0  0.562012  0.008108
1336  0.065217     0.0  0.264730  0.014144
1337  0.934783     1.0  0.352704  0.447249

[1338 rows x 4 columns]


In [7]:
#veriyi %70 eğitim, % 30 test olacak şekilde bölüyoruz
from sklearn.model_selection import train_test_split
train, test = train_test_split(normalized_df_new, test_size=0.3,random_state = 17) 

In [8]:
# label 'charges' olacak şekilde veriyi ayrıştırıyoruz
X_train = train.drop("charges",axis=1)
y_train = train.loc[:,"charges"]

X_test = test.drop("charges",axis=1)
y_test = test.loc[:,"charges"]

5. MODEL SEÇME

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures

In [10]:
linear_reg = LinearRegression()
linear_reg.fit(X_train,y_train)
linear_reg_pred = linear_reg.predict(X_test)
scores=cross_val_score(linear_reg,X_test,y_test,cv=10)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.70 accuracy with a standard deviation of 0.13


Polynomial Regresssion

In [11]:
#from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree=3)
# Belirlenen dereceye göre bağımsız değişkenler hazırlanır
poly_train = poly_reg.fit_transform(X_train)
poly_test =poly_reg.fit_transform(X_test)

L_regression = LinearRegression()

model = L_regression.fit(poly_train, y_train)

scores=cross_val_score(model,poly_test,y_test,cv=10)
print("%0.2f doğruluk (accuracy) ve standart sapması  %0.2f" % (scores.mean(), scores.std()))

0.82 doğruluk (accuracy) ve standart sapması  0.11


Supprt Vector Regression

In [12]:
svr_regressor = SVR(kernel = 'rbf')
svr_regressor.fit(X_train, y_train)
scores=cross_val_score(svr_regressor , X_test, y_test, cv=10)
print("%0.2f doğruluk (accuracy) ve standart sapması  %0.2f" % (scores.mean(), scores.std()))

0.67 doğruluk (accuracy) ve standart sapması  0.13


Decison tree Regression

In [24]:
dec_tree_reg=DecisionTreeRegressor(max_depth=3)
dec_tree_reg.fit(X_train,y_train)
scores=cross_val_score(dec_tree_reg, X_test, y_test, cv=10)
print("%0.2f doğruluk (accuracy) ve standart sapması %0.2f" % (scores.mean(), scores.std()))

0.84 doğruluk (accuracy) ve standart sapması 0.10


Random Forest Regressor

In [14]:
rr_regressor = RandomForestRegressor(n_estimators = 10)
rr_regressor.fit(X_train, y_train)
scores=cross_val_score(rr_regressor , X_test, y_test, cv=10)
print("%0.2f doğruluk (accuracy) ve standart sapması %0.2f" % (scores.mean(), scores.std()))

0.78 doğruluk (accuracy) ve standart sapması 0.11


In [ ]:
GradientBoostingRegressor

In [15]:
from sklearn.ensemble import GradientBoostingRegressor

gb_regressor = GradientBoostingRegressor()
gb_regressor.fit(X_train, y_train)
scores=cross_val_score(gb_regressor , X_test, y_test, cv=10)
print("%0.2f doğruluk (accuracy) ve standart sapması %0.2f" % (scores.mean(), scores.std()))

0.81 doğruluk (accuracy) ve standart sapması 0.10


Decision Tree Regressor 0.84 doğruluk (accuracy) ve standart sapması  0.10 ile en iyi sonucu verdi

6. HİPER-PARAMETRE OPTIMIZASYONU

In [29]:
# max_depthin  Decision-Tree Regresyona etkisi
for i in range(1,5):
    dec_tree_reg=DecisionTreeRegressor(max_depth=i)
    dec_tree_reg.fit(X_train,y_train)
    scores=cross_val_score(dec_tree_reg, X_test, y_test, cv=10)
    print(" max_depth = %d, %0.2f doğruluk (accuracy) ve standart sapması  %0.2f" % (max_depth,scores.mean(), scores.std()))

 max_depth = 1, 0.52 doğruluk (accuracy) ve standart sapması  0.14
 max_depth = 1, 0.80 doğruluk (accuracy) ve standart sapması  0.08
 max_depth = 1, 0.84 doğruluk (accuracy) ve standart sapması  0.10
 max_depth = 1, 0.82 doğruluk (accuracy) ve standart sapması  0.09


Gridsearch ile optimizasyon

In [38]:
reg=DecisionTreeRegressor()
params=reg.get_params().keys()
params

dict_keys(['ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'presort', 'random_state', 'splitter'])

In [47]:
from sklearn.model_selection import GridSearchCV
dtm = DecisionTreeRegressor(max_depth=4,
                           min_samples_split=5,
                           max_leaf_nodes=10)

dtm.fit(X_train,y_train)
param_grid = {"criterion": ["mse", "mae"],
              "min_samples_split": [10, 20, 40],
              "max_depth": [2, 6, 8],
              "min_samples_leaf": [20, 40, 100],
              "max_leaf_nodes": [5, 20, 100],
              }

## Comment in order to publish in kaggle.

grid_cv_dtm = GridSearchCV(dtm, param_grid, cv=10)

grid_cv_dtm.fit(X_train,y_train)

print(grid_cv_dtm.best_params_)

bestDTR = grid_cv_dtm.best_estimator_
print('Test accuracy: %.3f' % bestDTR.score(X_test, y_test))



{'criterion': 'mse', 'max_depth': 6, 'max_leaf_nodes': 20, 'min_samples_leaf': 20, 'min_samples_split': 10}
Test accuracy: 0.860


7. MODELİ DEĞERLENDİRME

In [48]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

dec_tree_reg=DecisionTreeRegressor(max_depth=6, max_leaf_nodes= 20,min_samples_leaf=20,min_samples_split=10)
dec_tree_reg.fit(X_train,y_train)
y_pred=dec_tree_reg.predict(X_test)

mse=mean_squared_error(y_test,y_pred)
print(" Mean Squared Error:" ,mse)

mae=mean_absolute_error(y_test,y_pred)
print(" Mean absolute Error:" ,mae)



 Mean Squared Error: 0.004335161854851345
 Mean absolute Error: 0.040288847916336634
